In [1]:
from openqaoa.qaoa_parameters import PauliOp, Hamiltonian, QAOACircuitParams, create_qaoa_variational_params
# from eqaoa.qaoa_parameters.baseparams import QAOACircuitParams
from openqaoa.backends.qpus.qpu_auth import AccessObjectQiskit
from openqaoa.backends.qaoa_backend import *
from openqaoa.optimizers import get_optimizer
from qiskit import QuantumCircuit
from openqaoa.utilities import *

In [2]:
cost_hamil = Hamiltonian([PauliOp('ZZ', (0, 1)), PauliOp('ZZ', (1, 2)), PauliOp('ZZ', (0, 2))], 
                         [1, 1, 1], 1)

In [3]:
mixer_hamil = X_mixer_hamiltonian(n_qubits=3)
mixer_hamil.expression

-1X_{0} - 1X_{1} - 1X_{2} + 0

In [4]:
circuit_params = QAOACircuitParams(cost_hamil, mixer_hamil, p=2)

In [5]:
variate_params = create_qaoa_variational_params(circuit_params,'standard','ramp')

In [6]:
auth_dict = { "QISKIT": 
             { "API_TOKEN": "API TOKEN",
              "HUB": "ibm-q", 
              "GROUP": "open", 
              "PROJECT": "main"}
            }

access_obj = AccessObjectQiskit(auth_dict["QISKIT"]["API_TOKEN"], auth_dict["QISKIT"]["HUB"], 
                   auth_dict["QISKIT"]["GROUP"], auth_dict["QISKIT"]["PROJECT"], 'ibmq_qasm_simulator')

In [7]:
backend_cloud = get_qaoa_backend(circuit_params, access_obj, n_shots=1000)

In [8]:
backend_cloud.parametric_circuit.draw()

┌───┐                                             »
  q0_0: ┤ H ├──■──────────────────────────────────■───────»
        ├───┤┌─┴─┐┌────────────────────────────┐┌─┴─┐     »
  q0_1: ┤ H ├┤ X ├┤ Rz(2*['2q', 'cost', 0, 0]) ├┤ X ├──■──»
        ├───┤└───┘└────────────────────────────┘└───┘┌─┴─┐»
  q0_2: ┤ H ├────────────────────────────────────────┤ X ├»
        └───┘                                        └───┘»
meas: 3/══════════════════════════════════════════════════»
                                                          »
«                                                                              »
«  q0_0: ─────────────────────────────────────■────────────────────────────────»
«                                             │  ┌───────────────────────────┐ »
«  q0_1: ────────────────────────────────■────┼──┤ Rx(['1q', 'mixer', 0, 1]) ├─»
«        ┌────────────────────────────┐┌─┴─┐┌─┴─┐├───────────────────────────┴┐»
«  q0_2: ┤ Rz(2*['2q', 'cost', 0, 1]) ├┤ X ├┤ X ├┤ Rz(2*['2q', 'cost', 0, 2]) ├»
«        └────────────────────────────┘└───┘└───┘└────────────────────────────┘»
«meas: 3/══════════════════════════════════════════════════════════════════════»
«                                                                              »
«             ┌───────────────────────────┐                                   »
«  q0_0: ──■──┤ Rx(['1q', 'mixer', 0, 0]) ├──■────────────────────────────────»
«          │  └───────────────────────────┘┌─┴─┐┌────────────────────────────┐»
«  q0_1: ──┼───────────────────────────────┤ X ├┤ Rz(2*['2q', 'cost', 1, 0]) ├»
«        ┌─┴─┐┌───────────────────────────┐└───┘└────────────────────────────┘»
«  q0_2: ┤ X ├┤ Rx(['1q', 'mixer', 0, 2]) ├───────────────────────────────────»
«        └───┘└───────────────────────────┘                                   »
«meas: 3/═════════════════════════════════════════════════════════════════════»
«                                                                             »
«                                                          »
«  q0_0: ──■────────────────────────────────────────────■──»
«        ┌─┴─┐                                          │  »
«  q0_1: ┤ X ├──■──────────────────────────────────■────┼──»
«        └───┘┌─┴─┐┌────────────────────────────┐┌─┴─┐┌─┴─┐»
«  q0_2: ─────┤ X ├┤ Rz(2*['2q', 'cost', 1, 1]) ├┤ X ├┤ X ├»
«             └───┘└────────────────────────────┘└───┘└───┘»
«meas: 3/══════════════════════════════════════════════════»
«                                                          »
«                                           ┌───────────────────────────┐ ░ ┌─┐»
«  q0_0: ────────────────────────────────■──┤ Rx(['1q', 'mixer', 1, 0]) ├─░─┤M├»
«        ┌───────────────────────────┐   │  └───────────────────────────┘ ░ └╥┘»
«  q0_1: ┤ Rx(['1q', 'mixer', 1, 1]) ├───┼────────────────────────────────░──╫─»
«        ├───────────────────────────┴┐┌─┴─┐┌───────────────────────────┐ ░  ║ »
«  q0_2: ┤ Rz(2*['2q', 'cost', 1, 2]) ├┤ X ├┤ Rx(['1q', 'mixer', 1, 2]) ├─░──╫─»
«        └────────────────────────────┘└───┘└───────────────────────────┘ ░  ║ »
«meas: 3/════════════════════════════════════════════════════════════════════╩═»
«                                                                            0 »
«              
«  q0_0: ──────
«        ┌─┐   
«  q0_1: ┤M├───
«        └╥┘┌─┐
«  q0_2: ─╫─┤M├
«         ║ └╥┘
«meas: 3/═╩══╩═
«         1  2

In [10]:
backend_cloud.qaoa_circuit(variate_params).draw()

┌───┐                                                                 »
  q0_0: ┤ H ├──■─────────────────────────────■────────────────────────────────»
        ├───┤┌─┴─┐┌───────────────────────┐┌─┴─┐                              »
  q0_1: ┤ H ├┤ X ├┤ Rz(0.350000000000000) ├┤ X ├──■───────────────────────────»
        ├───┤└───┘└───────────────────────┘└───┘┌─┴─┐┌───────────────────────┐»
  q0_2: ┤ H ├───────────────────────────────────┤ X ├┤ Rz(0.350000000000000) ├»
        └───┘                                   └───┘└───────────────────────┘»
meas: 3/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                                                 ┌────────────────────────┐»
«  q0_0: ───────■──────────────────────────────■──┤ Rx(-0.525000000000000) ├»
«               │  ┌────────────────────────┐  │  └────────────────────────┘»
«  q0_1: ──■────┼──┤ Rx(-0.525000000000000) ├──┼────────────────────────────»
«        ┌─┴─┐┌─┴─┐├───────────────────────┬┘┌─┴─┐┌────────────────────────┐»
«  q0_2: ┤ X ├┤ X ├┤ Rz(0.350000000000000) ├─┤ X ├┤ Rx(-0.525000000000000) ├»
«        └───┘└───┘└───────────────────────┘ └───┘└────────────────────────┘»
«meas: 3/═══════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                                            »
«  q0_0: ──■────────────────────────────■────────────────────────────────────»
«        ┌─┴─┐┌──────────────────────┐┌─┴─┐                                  »
«  q0_1: ┤ X ├┤ Rz(1.05000000000000) ├┤ X ├──■────────────────────────────■──»
«        └───┘└──────────────────────┘└───┘┌─┴─┐┌──────────────────────┐┌─┴─┐»
«  q0_2: ──────────────────────────────────┤ X ├┤ Rz(1.05000000000000) ├┤ X ├»
«                                          └───┘└──────────────────────┘└───┘»
«meas: 3/════════════════════════════════════════════════════════════════════»
«                                                                            »
«                                            ┌────────────────────────┐ ░ ┌─┐»
«  q0_0: ──■──────────────────────────────■──┤ Rx(-0.175000000000000) ├─░─┤M├»
«          │  ┌────────────────────────┐  │  └────────────────────────┘ ░ └╥┘»
«  q0_1: ──┼──┤ Rx(-0.175000000000000) ├──┼─────────────────────────────░──╫─»
«        ┌─┴─┐└┬──────────────────────┬┘┌─┴─┐┌────────────────────────┐ ░  ║ »
«  q0_2: ┤ X ├─┤ Rz(1.05000000000000) ├─┤ X ├┤ Rx(-0.175000000000000) ├─░──╫─»
«        └───┘ └──────────────────────┘ └───┘└────────────────────────┘ ░  ║ »
«meas: 3/══════════════════════════════════════════════════════════════════╩═»
«                                                                          0 »
«              
«  q0_0: ──────
«        ┌─┐   
«  q0_1: ┤M├───
«        └╥┘┌─┐
«  q0_2: ─╫─┤M├
«         ║ └╥┘
«meas: 3/═╩══╩═
«         1  2

In [7]:
backend_qiskit_statevec = get_qaoa_backend(circuit_params, 'qiskit_statevec_simulator')

In [8]:
print(backend_qiskit_statevec.expectation(variate_params))
print(backend_qiskit_statevec.expectation_w_uncertainty(variate_params))

0.23595157051912216
(0.23595157051912216, 0.9424081591571926)


In [10]:
backend_qiskit_statevec.qfim(variate_params)

/home/vishal/codes/openqaoa-temporary/openqaoa/basebackend.py:525: ComplexWarning: Casting complex values to real discards the imaginary part
  qfim_array[i][j] = np.real( np.vdot(di_psi, dj_psi) ) - np.vdot(di_psi, psi)*np.vdot(psi, dj_psi)


array([[ 0.35549347,  0.35520208,  0.09094093, -0.71718402],
       [ 0.35520208,  2.74634507,  2.75894955,  0.40636562],
       [ 0.09094093,  2.75894955,  3.        ,  1.06940255],
       [-0.71718402,  0.40636562,  1.06940255,  1.97418693]])

In [12]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q-startup','entropica-labs','reservations')
backend = provider.get_backend('ibmq_guadalupe')
noise_model = NoiseModel.from_backend(backend)

In [13]:
backend_qiskit_shot = get_qaoa_backend(circuit_params, 'qiskit_shot_simulator', n_shots=100)#, noise_model=noise_model)

In [14]:
print(backend_qiskit_shot.expectation(variate_params))
print(backend_qiskit_shot.expectation_w_uncertainty(variate_params))

0.21875
(0.21875, 1.268226492981439)


In [15]:
current_ckt = backend_qiskit_shot.qaoa_circuit(variate_params)

In [16]:
#print parametric qaoa circuit
backend_qiskit_shot.parametric_circuit.draw()

┌───┐                                                      »
  q1_0: ┤ H ├─■────────────────────────────────────────────────────»
        ├───┤ │ZZ(['2q', 'cost', 0, 0])                            »
  q1_1: ┤ H ├─■──────────────────────────■─────────────────────────»
        ├───┤                            │ZZ(['2q', 'cost', 0, 1]) »
  q1_2: ┤ H ├────────────────────────────■─────────────────────────»
        └───┘                                                      »
meas: 3/═══════════════════════════════════════════════════════════»
                                                                   »
«                                   ┌───────────────────────────┐»
«  q1_0: ─■─────────────────────────┤ Rx(['1q', 'mixer', 0, 0]) ├»
«         │                         ├───────────────────────────┤»
«  q1_1: ─┼─────────────────────────┤ Rx(['1q', 'mixer', 0, 1]) ├»
«         │ZZ(['2q', 'cost', 0, 2]) ├───────────────────────────┤»
«  q1_2: ─■─────────────────────────┤ Rx(['1q', 'mixer', 0, 2]) ├»
«                                   └───────────────────────────┘»
«meas: 3/════════════════════════════════════════════════════════»
«                                                                »
«                                                              »
«  q1_0: ─■────────────────────────────────────────────────────»
«         │ZZ(['2q', 'cost', 1, 0])                            »
«  q1_1: ─■──────────────────────────■─────────────────────────»
«                                    │ZZ(['2q', 'cost', 1, 1]) »
«  q1_2: ────────────────────────────■─────────────────────────»
«                                                              »
«meas: 3/══════════════════════════════════════════════════════»
«                                                              »
«                                   ┌───────────────────────────┐ ░ ┌─┐      
«  q1_0: ─■─────────────────────────┤ Rx(['1q', 'mixer', 1, 0]) ├─░─┤M├──────
«         │                         ├───────────────────────────┤ ░ └╥┘┌─┐   
«  q1_1: ─┼─────────────────────────┤ Rx(['1q', 'mixer', 1, 1]) ├─░──╫─┤M├───
«         │ZZ(['2q', 'cost', 1, 2]) ├───────────────────────────┤ ░  ║ └╥┘┌─┐
«  q1_2: ─■─────────────────────────┤ Rx(['1q', 'mixer', 1, 2]) ├─░──╫──╫─┤M├
«                                   └───────────────────────────┘ ░  ║  ║ └╥┘
«meas: 3/════════════════════════════════════════════════════════════╩══╩══╩═
«                                                                    0  1  2

In [17]:
#print the full circuit with attached angles
current_ckt.draw()

┌───┐                                                »
  q1_0: ┤ H ├─■──────────────────────────────────────────────»
        ├───┤ │ZZ(0.175000000000000)                         »
  q1_1: ┤ H ├─■───────────────────────■──────────────────────»
        ├───┤                         │ZZ(0.175000000000000) »
  q1_2: ┤ H ├─────────────────────────■──────────────────────»
        └───┘                                                »
meas: 3/═════════════════════════════════════════════════════»
                                                             »
«                                ┌────────────────────────┐»
«  q1_0: ─■──────────────────────┤ Rx(-0.525000000000000) ├»
«         │                      ├────────────────────────┤»
«  q1_1: ─┼──────────────────────┤ Rx(-0.525000000000000) ├»
«         │ZZ(0.175000000000000) ├────────────────────────┤»
«  q1_2: ─■──────────────────────┤ Rx(-0.525000000000000) ├»
«                                └────────────────────────┘»
«meas: 3/══════════════════════════════════════════════════»
«                                                          »
«                                                        »
«  q1_0: ─■──────────────────────────────────────────────»
«         │ZZ(0.525000000000000)                         »
«  q1_1: ─■───────────────────────■──────────────────────»
«                                 │ZZ(0.525000000000000) »
«  q1_2: ─────────────────────────■──────────────────────»
«                                                        »
«meas: 3/════════════════════════════════════════════════»
«                                                        »
«                                ┌────────────────────────┐ ░ ┌─┐      
«  q1_0: ─■──────────────────────┤ Rx(-0.175000000000000) ├─░─┤M├──────
«         │                      ├────────────────────────┤ ░ └╥┘┌─┐   
«  q1_1: ─┼──────────────────────┤ Rx(-0.175000000000000) ├─░──╫─┤M├───
«         │ZZ(0.525000000000000) ├────────────────────────┤ ░  ║ └╥┘┌─┐
«  q1_2: ─■──────────────────────┤ Rx(-0.175000000000000) ├─░──╫──╫─┤M├
«                                └────────────────────────┘ ░  ║  ║ └╥┘
«meas: 3/══════════════════════════════════════════════════════╩══╩══╩═
«                                                              0  1  2

In [18]:
backend_cloud.expectation_w_uncertainty(variate_params)

NameError: name 'backend_cloud' is not defined

In [ ]:
optimizer_dict = {'method': 'cobyla', 'maxiter': 10}

In [ ]:
optimizer_obj = get_optimizer(backend_qiskit_shot, variate_params, optimizer_dict)

In [ ]:
optimizer_obj

Optimizer for VQA of type: QAOABaseBackendShotBased 
Backend: QAOAQiskitBackendShotBasedSimulator 
Method: COBYLA with Max Iterations: 10

In [ ]:
optimizer_obj()
optimizer_obj.results_information()

{'opt result':      fun: 0.16796875
    maxcv: 0.0
  message: 'Maximum number of function evaluations has been exceeded.'
     nfev: 10
   status: 2
  success: False
        x: array([ 0.23860891, -0.49749158, -0.55008868,  0.57481587]),
 'parameter log': [[0.5249999999999999, 0.175, 0.175, 0.5249999999999999],
  [1.525, 0.175, 0.175, 0.5249999999999999],
  [0.5249999999999999, 1.175, 0.175, 0.5249999999999999],
  [0.5249999999999999, 0.175, 1.175, 0.5249999999999999],
  [0.5249999999999999, 0.175, 0.175, 1.525],
  [0.4285719711458033,
   -0.4974915778532932,
   -0.5500886845010369,
   0.4122919143262636],
  [0.31475014759370734,
   -1.2912879317166122,
   0.03234817895156872,
   0.27925341926537217],
  [0.3133479216391065,
   -0.24268654978312187,
   -0.9420833244246505,
   0.2776144538638907],
  [0.23860890950689823,
   -0.4974915778532932,
   -0.5500886845010369,
   0.5748158670617713],
  [0.531542560099041,
   -0.5287021363367402,
   -0.6856147964570463,
   0.955405475916242]],
 'f

In [ ]:
optimizer_obj()
optimizer_obj.results_information()

{'opt result':      fun: 0.1171875
    maxcv: 0.0
  message: 'Maximum number of function evaluations has been exceeded.'
     nfev: 10
   status: 2
  success: False
        x: array([ 0.49041297, -0.5005708 , -0.54492259,  0.23068843]),
 'parameter log': [[0.5249999999999999, 0.175, 0.175, 0.5249999999999999],
  [1.525, 0.175, 0.175, 0.5249999999999999],
  [0.5249999999999999, 1.175, 0.175, 0.5249999999999999],
  [0.5249999999999999, 0.175, 1.175, 0.5249999999999999],
  [0.5249999999999999, 0.175, 0.175, 1.525],
  [0.4285719711458033,
   -0.4974915778532932,
   -0.5500886845010369,
   0.4122919143262636],
  [0.31475014759370734,
   -1.2912879317166122,
   0.03234817895156872,
   0.27925341926537217],
  [0.3133479216391065,
   -0.24268654978312187,
   -0.9420833244246505,
   0.2776144538638907],
  [0.23860890950689823,
   -0.4974915778532932,
   -0.5500886845010369,
   0.5748158670617713],
  [0.531542560099041,
   -0.5287021363367402,
   -0.6856147964570463,
   0.955405475916242],
  [0.

In [ ]:
# derivative_function move to QPUBackend Template class